# 检测环境

In [ ]:
import os

def check_environment(output):
    environments = {
        "AutoDL": {
            "content_path": "/root/autodl-tmp/content",
            "env_name": "AutoDL"
        },
        "OpenBayes": {
            "content_path": "/openbayes/home/content",
            "env_name": "OpenBayes"
        }
    }
    # 检查输出是否包含字符串"autodl"或"openbayes"
    # for i in output:
    if "AutoDL" in output:
        return environments["AutoDL"]
    elif "OpenBayes" in output:
        return environments["OpenBayes"]
    # 如果输出中没有包含上述字符串，则返回空字典
    return {}


def detect_environment():
    # 初始化 content_path 和 env_name 变量
    content_path = None
    env_name = None
    
    commands = []
    
    # 根据条件把命令行存储在列表中
    if os.path.exists('/openbayes/home'):
        commands.append("cd /openbayes/home && chmod +x /etc/welcome && /etc/welcome")
    
    if os.path.exists('/root'):
        commands.append("chmod +x /etc/autodl-motd && /etc/autodl-motd")

    # 遍历命令行列表，执行命令并存储输出
    for command in commands:
        try:
            import subprocess
            # 使用 subprocess 模块执行命令
            r = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
            output = r.stdout
            # 将输出转换为字符串
            output = output.decode()
            # print(output)
            result = check_environment(output)
            content_path = result["content_path"]
            env_name = result["env_name"]
            # 如果检测到环境，则退出循环
            if content_path and env_name:
                break
        except Exception as e:
            # print("无法执行命令：", e)
            continue

    # 打印结果
    if content_path and env_name:
        print("当前运行环境：", env_name)
        print("内容路径：", content_path)
    else:
        print("未检测到当前运行环境")
        
    return {
        "content_path":content_path,
        "env_name":env_name
    }

# Ping

In [ ]:
import os
import threading
import time
import requests

def print_time(start_time):
    time_elapsed = time.time() - start_time
    minutes = int(time_elapsed / 60)
    seconds = int(time_elapsed % 60)
    millis = int((time_elapsed % 1) * 1000)
    message = f'\033[1;33mPing程序运行耗时\033[0m: \033[1;32m'
    if minutes > 0:
        message += f'{minutes}分'
    if seconds > 0 or minutes > 0:
        message += f'{seconds}秒'
    message += f'{millis}毫秒\033[0m'
    print(message)

def print_sentence(proxy_info):
    region = proxy_info['region']
    ip = proxy_info['ip']
    port = proxy_info['port']

    print(f"Ping通该IP与端口：{ip}:{port}，正在使用【{region}】代理")

def test_proxy(item):
    ip = item['ip']
    port = item['port']
    http_proxy = f'http://{ip}:{port}'
    https_proxy = f'https://{ip}:{port}'
    socks_proxy = f'socks5://{ip}:{port}'
    proxies = {
        'http': http_proxy,
        'https': https_proxy,
        'socks5': socks_proxy,
    }
    for protocol in proxies.keys():
        proxy = proxies[protocol]
        try:
            response = requests.get('http://www.gstatic.com/generate_204', proxies={protocol: proxy}, timeout=3)
            if response.status_code == 204:
                return True
        except:
            pass
    return False

def ping_threading(ipDict):
    start_time = time.time()
    # 创建一个事件，用于在成功 ping 通某个 IP 地址后通知其他线程退出
    event = threading.Event()

    # 存储成功 ping 通的 IP 地址
    result = []

    # 为每个 IP 地址创建一个线程，用于 ping 该地址
    for item in ipDict:
        thread = threading.Thread(target=ping_ip, args=(item, result, event))
        thread.start()

    # 如果有线程成功 ping 通了某个 IP 地址，则通知其他线程退出
    if event.wait():
        print_time(start_time)
        print_sentence(result[0])
        # 返回成功 ping 通的 IP 地址
        return result[0]

def ping_ip(item, result, event):
    # 测试代理是否可用
    if test_proxy(item):
        # 如果代理可用，则将该地址添加到结果列表中
        result.append(item)
        # 通知其他线程退出
        event.set()

# 挂代理

In [ ]:
import os

de = detect_environment()
env_name, content_path = de['env_name'], de['content_path']
    
ipDict = [
    {'region': '自供', 'ip': 'localhost', 'port': '7779'},
]

if env_name == 'AutoDL':
    ipDict += [
        {'region': '芜湖', 'ip': '192.168.0.91', 'port': '12798'},
        {'region': '北京', 'ip': '100.72.64.19', 'port': '12798'},
        {'region': '内蒙', 'ip': '192.168.1.174', 'port': '12798'},
        {'region': '泉州', 'ip': '10.55.146.88', 'port': '12798'},
        {'region': '南京', 'ip': '172.181.217.43', 'port': '12798'},
        {'region': '佛山', 'ip': '192.168.126.12', 'port': '12798'},
        {'region': 'A100专区', 'ip': '172.31.1.127', 'port': '12798'},
    ]
elif env_name == 'OpenBayes':
    ipDict += [
        {'region': '贝式', 'ip': 'alchemist-experience', 'port': '7890'},
    ]

ipDict += [
    {'region': '九天', 'ip': '172.22.17.74', 'port': '3928'},
]


debugging=False

import urllib.request

def proxyWget(url, out, proxyURL):
    # 设置代理
    proxy = urllib.request.ProxyHandler({'http': proxyURL})
    opener = urllib.request.build_opener(proxy)
    urllib.request.install_opener(opener)

    # 下载文件
    file_path, _ = urllib.request.urlretrieve(url, out)

    # 返回已完成下载的文件路径
    return file_path

def getProxyURL(ip,port):
    return f'http://{ip}:{port}'


def autoRegion():
    # import ping_threading  
    global region, proxy, proxyURL
    cb = ping_threading(ipDict)
    # print(cb)
    region = cb['region']
    ip = cb['ip']
    port = cb['port']
    proxy = makeCLI(ip,port)
    proxyURL = getProxyURL(ip,port)

def makeCLI(ip,port):
    return f'export http_proxy={getProxyURL(ip,port)} && export https_proxy={getProxyURL(ip,port)}'


def setProxyCLI():
    autoRegion()
    return({'region': region, 'proxy': proxy, 'proxyURL': proxyURL})


def setProxy():

    import os
    import subprocess
    from ipywidgets import interact, widgets
    from IPython.display import display, clear_output

    global proxyURL
    btnArray = []


    def printSuccess(region):
        # print(f'已挂载【{region}】对应的代理')
        picked.button_style = 'info'
        picked.description = f'在使用【{region}】代理'
        # picked.icon = 'check'

    def printFail():
        # print(f'已挂载【{region}】对应的代理')
        picked.button_style = 'warning'
        picked.description = f'没有合适的代理'
        picked.icon = 'exclamation-triangle'

    def autoClick(region):
        for o in btnArray:  # 取消所有按钮被选中的样式
            o.button_style = ''
            o.icon = ''
        printSuccess(region)
        for k in btnArray:
            if k.description == region:
                k.button_style = 'success'
                k.icon = 'check'

    def btn_eventhandle(obj):
        global region, proxy
        for o in btnArray:  # 取消所有按钮被选中的样式
            o.button_style = ''
            o.icon = ''
        obj.button_style = 'success'
        obj.icon = 'check'
        region = obj.description
        ip = ipDict[region]
        proxy = makeCLI(ip)
        printSuccess(obj.description)

    picked = widgets.Button(
        value=False,
        description=f'自动侦测中...',
        disabled=False,
        button_style='warning',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='请点击下方的区名',
        icon='question-circle'  # (FontAwesome names without the `fa-` prefix)
    )

    for i in ipDict:
        btn = widgets.Button(
            description=i['region'],
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            # icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        btn.on_click(btn_eventhandle)
        btnArray.append(btn)

    btnArray.insert(0, picked)

    for e in btnArray:
        display(e)

    autoRegion()
    if debugging==False:
        clear_output(wait=True)
    
    if region != '未知':
        autoClick(region)
    else:
        if debugging==False:
            clear_output(wait=True)
        printFail()

    for e in btnArray:
        display(e)

    return({'region': region, 'proxy': proxy, 'proxyURL': proxyURL})

def doProxy(force=False):
    global proxy,region
    if force:
        sp = setProxy()
        proxy = sp['proxy']
        region = sp['region']
        proxyURL = sp['proxyURL']
    else:
        try:
            proxy,region
        except NameError:
            sp = setProxy()
            proxy = sp['proxy']
            region = sp['region']
            proxyURL = sp['proxyURL']
            
def selProxy(force=False,region_name=None):
    # 如果指定了region_name，剔除不符合要求的代理
    global ipDict
    if region_name is not None:
        ipDict = [ip for ip in ipDict if ip['region'] == region_name]
    doProxy(force=force)